In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

/tmp/ipykernel_55161/2129287977.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-13 15:31:09.129200: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 15:31:09.130107: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 15:31:09.580996: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable 

In [2]:
prompt_embeddings = np.load('../processed_data/embedding_summary.npy')

In [3]:
movie_embeddings = np.load('../processed_data/embeddings_plot.npy')

In [7]:
movies_df = pd.read_pickle('../raw_data/title_plot_movieId.pkl')

In [8]:
movies_embed = pd.DataFrame(movie_embeddings, index= movies_df['title'])
prompts_embed = pd.DataFrame(prompt_embeddings, index= movies_df['title'])

In [11]:
prompts_embed.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
title,,,,,,,,,,,,,,,,,,,,,
Mr. Holland's Opus,-0.643445,-0.395430,-4.088519,-2.476784,0.065044,-1.401066,-0.521427,-0.292286,-1.009134,-0.137444,...,-0.562036,-0.887493,-0.582381,0.349723,0.947146,-1.402179,-0.538435,-2.477199,0.457515,1.923286
Scarface,-0.667590,-0.706178,-4.087876,-2.354048,0.272632,-1.616781,-0.838146,-0.328010,-0.813613,0.347498,...,-0.254029,-0.721833,-1.064592,0.780748,1.435769,-1.081843,-0.111125,-2.541416,0.316348,2.190334
Flightplan,-0.331173,-0.430575,-4.064622,-2.678680,-0.466110,-1.113054,-1.040370,0.071133,-0.780748,0.716909,...,-0.671378,-0.673283,-0.659972,-0.042420,0.875326,-1.276242,-0.566479,-2.622893,0.285767,2.197447
Little Caesar,-0.576120,-0.832623,-3.914762,-2.431295,0.223214,-1.759424,-0.891972,-0.448007,-0.782553,0.160594,...,-0.559849,-0.868267,-1.135357,0.498278,1.406906,-1.143107,-0.104455,-2.373625,0.202973,2.104166
Savages,-0.601033,-0.968912,-4.179355,-2.301611,0.613752,-1.425504,-1.224570,0.055443,-0.931644,0.525285,...,0.012788,-0.507534,-1.144119,0.681214,1.422258,-1.281380,-0.127091,-2.888142,0.222285,2.507713


In [9]:
movies_df.head()

,title,plot_synopsis,movieId
0,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...",62
1,Scarface,"In May 1980, a Cuban man named Tony Montana (A...",4262
5,Flightplan,Kyle Pratt (Jodie Foster) is a propulsion engi...,37727
6,Little Caesar,Small-time Italian-American criminals Caesar E...,25773
7,Savages,The movie begins with a video being shot of me...,56956


In [24]:
def merge_prompt_with_favorites(prompts_embedded: pd.DataFrame, favs: list) -> pd.DataFrame:
    # get the embedded data
    # TODO: Adjust to latest models
    # Load titles
    filepath_title = "../raw_data/movie_title.pkl"
    titles = pd.read_pickle(filepath_title)

    # Load embedded plots
    plot_embedded = movies_embed

    df_filtered = plot_embedded[plot_embedded.index.isin(favs)] # embedded dataframe with just the favorites
    series = prompts_embedded.iloc[0,:] # convert the prompt dataframe to a series
    df_filtered.loc['prompt'] = series.to_list() # add the prompt to the dataframe (concat didnt work well)
    mean_df = df_filtered.mean(axis=0).to_frame().T # get the mean of the dataframe, keep it as dataframe
    mean_df.index = ['prompt'] # set the index to 'prompt'
    return mean_df

In [25]:
merge_prompt_with_favorites(prompts_embed, ['Scarface'])

/tmp/ipykernel_55161/1646288669.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc['prompt'] = series.to_list() # add the prompt to the dataframe (concat didnt work well)


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
prompt,-1.274325,-0.630273,-4.340691,-1.686134,0.231956,-0.780448,-0.099199,0.586124,-0.89807,-0.183438,...,-0.293494,-0.34694,-0.860379,-0.07812,0.591207,-0.205663,-1.118057,-1.324238,-0.397963,1.713984


In [34]:
def merge_prompt_with_favorites(prompt_vector, prompt_bert: pd.DataFrame, favs: list) -> pd.DataFrame:
    # get the embedded data
    # TODO: Adjust to latest models
    # Load titles
    filepath_title = "../raw_data/movie_title.pkl"
    titles = pd.read_pickle(filepath_title)

    # Load embedded plots
    plot_embedded = movies_embed

    df_filtered = plot_embedded[plot_embedded.index.isin(favs)] # embedded dataframe with just the favorites
    series = prompt_bert.iloc[0,:] # convert the prompt dataframe to a series
    df_filtered.loc['prompt'] = series.to_list() # add the prompt to the dataframe (concat didnt work well)
    mean_df_bert = df_filtered.mean(axis=0).to_frame().T # get the mean of the dataframe, keep it as dataframe
    mean_df_bert.index = ['prompt'] # set the index to 'prompt'

    filepath_matrix = '../processed_data/vectorized_summaries.pkl'
    tf_idf_matrix = pd.read_pickle(filepath_matrix)
    matrix_df = pd.DataFrame(tf_idf_matrix.toarray(), index = titles.values)

    matrix_filtered = matrix_df[matrix_df.index.isin(favs)]
    series = prompt_vector.iloc[0,:] # convert the prompt dataframe to a series
    matrix_filtered.loc['prompt'] = series.to_list() # add the prompt to the dataframe (concat didnt work well)
    mean_df_vector = matrix_filtered.mean(axis=0).to_frame().T # get the mean of the dataframe, keep it as dataframe
    mean_df_vector.index = ['prompt'] # set the index to 'prompt'

    return mean_df_bert, mean_df_vector

In [32]:
filepath_matrix = '../processed_data/vectorized_summaries.pkl'
tf_idf_matrix = pd.read_pickle(filepath_matrix)
vector_df = pd.DataFrame(tf_idf_matrix, index = movies_df['title'])

In [35]:
merge_prompt_with_favorites(vector_df, prompts_embed, ['Scarface'])

/tmp/ipykernel_55161/2374377759.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc['prompt'] = series.to_list() # add the prompt to the dataframe (concat didnt work well)


ValueError: cannot set a row with mismatched columns